# Imports and functions

In [ ]:
import xarray as xr
import warnings
warnings.filterwarnings("ignore", category=xr.SerializationWarning)
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
from config import FLUX_DATA_PATH, FLUX_METADATA, MICASA_PREPROCESSED_DATA, MERRA_DATA_PATH

In [ ]:
from utils.functions import import_flux_metadata, import_site_RMSE_data

In [ ]:
from utils.plotting import polyfit1d_and_plot

# Open and combine virtual datasets

In [ ]:
# group, var = ["slv", "T2M"]
group, var = ["flx", "PRECTOTCORR"]

In [ ]:
var_avg=f"{var}_avg"

In [ ]:
MERRA2_store = f"M2store_tavgM_{group}_1991_2021"

In [ ]:
# Virtualized dataset path
ref_path = os.path.join(MERRA_DATA_PATH, MERRA2_store)

In [ ]:
ref_url1 = f"reference::{os.path.join(ref_path, "vstore1.parquet")}"
ref_url2 = f"reference::{os.path.join(ref_path, "vstore2.parquet")}"

In [ ]:
ds1 = xr.open_dataset(ref_url1, engine="zarr", consolidated=False)

In [ ]:
ds2 = xr.open_dataset(ref_url2, engine="zarr", consolidated=False)

In [ ]:
ds2[var]

In [ ]:
ds_combined = xr.concat([ds1[var],ds2[var]], dim="time")

In [ ]:
# Replace missing values with NaN
ds_combined = ds_combined.where(ds_combined!=999999986991104, np.nan)

In [ ]:
# ds_combined

# Import RSME values

In [ ]:
df_ANN = import_site_RMSE_data(FLUX_METADATA, '../analysis/RMSE_results_ANN.csv')

In [ ]:
df_ANN.head()

# Total 30 year averages

In [ ]:
ds_totavg = ds_combined.mean("time")

In [ ]:
ds_totavg

### Extract subset of values from xarray dataset

In [ ]:
# I should move this function to utils (this could speed up preprocessing?)
def nearest_da(df, da):
    """
    Vectorized extraction (more efficient for large datasets)
    """
    # Create arrays of lat/lon values
    lats = df['lat'].values
    lons = df['lon'].values
    
    # Use xarray's advanced indexing
    selected = da.sel(
        lat=xr.DataArray(lats, dims='points'),
        lon=xr.DataArray(lons, dims='points'),
        method='nearest'
    )
    
    return selected

In [ ]:
nearest_da(df_ANN, ds_totavg)

In [ ]:
df_ANN_tot = df_ANN.copy()
df_ANN_tot[var_avg] = nearest_da(df_ANN, ds_totavg).values

In [ ]:
df_ANN_tot.head()

In [ ]:
xlabel = '2-meter Temperature (K)\n30-Yr Average (1991-2021)'

In [ ]:
polyfit1d_and_plot(df_ANN_tot, f"{var}_avg", "NEE_RMSE", xlabel, f"NEE RSME vs MERRA2 {var}");

# Selective Year Average
### (Only average across years for each site from "Years of AmeriFlux data column")

In [ ]:
df_ANN = df_ANN.rename(columns = {'Years of AmeriFlux FLUXNET Data' : "YEARS_DATA"})

In [ ]:
def parse_years(year_string):
    if pd.isna(year_string): 
        return []
    return [int(year.strip()) for year in year_string.split(',')]

In [ ]:
df_ANN["YEARS_DATA"] = df_ANN["YEARS_DATA"].apply(parse_years)

In [ ]:
df_ANN.head()

In [ ]:
ds_annavgs = ds_combined.groupby("time.year").mean("time")

In [ ]:
test_ds = nearest_da(df_ANN, ds_annavgs)

In [ ]:
test_ds

### Create a mask to select only years of interest for each lat/lon

In [ ]:
mask = np.zeros_like(test_ds, dtype=bool)
mask.shape

In [ ]:
for i, yrs in enumerate(df_ANN["YEARS_DATA"]):
    test = np.isin(test_ds.year, yrs)
    mask[np.isin(test_ds.year, yrs), i] = True
    
means = test_ds.where(mask).mean('year')

In [ ]:
df_ANN_sub = df_ANN.copy()
df_ANN_sub[var_avg] = means.values

In [ ]:
df_ANN_sub.head()

In [ ]:
xlabel = '2-meter Temperature (K)\nAveraged Across Years of Available Data'

In [ ]:
polyfit1d_and_plot(df_ANN_sub, var_avg, "NEE_RMSE", xlabel, f"NEE RSME vs MERRA2 {var}");